# Perfect runbook

## Loading libraries and setting parameters

In [1]:
import sys

sys.path.append("../../..")

from src.common import helpers, config
from src.models.perfect import Perfect
from src.evaluation.evaluator import Evaluator
%load_ext autoreload

In [2]:
MODEL_NAME = "perfect_cf_d3"

paths = config.Paths(
    dataset_name="jobs_published",
    target_users_name="all",
    model_name=MODEL_NAME,
)

In [3]:
N_RECOMMENDATIONS = 10

## Model training

In [4]:
%autoreload
from src.models.perfect import Perfect

model = Perfect(path_test = paths.test,
               restrict_items_to_train = True,
               restrict_items_to_d3 = True,)

model.load_interactions(paths.train_and_validation)

model.preprocess()

model.fit()

## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS,
)

model.save_recommendations(recommendations, paths.recommendations)

619389it [1:54:00, 90.55it/s]  


## Evaluation

In [6]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.test,
    k=10,
    models_to_evaluate=[MODEL_NAME],
)

evaluator.prepare()

evaluator.evaluate_models()

619389it [00:44, 13942.44it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
perfect,0.518626,0.74645,0.500794,0.489366,0.896136,0.873285,0.954321,0.873221,0.954321,0.518626,0.880999,10.860545,0.682072,0.045679,0.696553
